In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import glob
from pyvis.network import Network
import numpy as np

In [2]:
PATH = [] # ALL excel sheets of different models to be found here
MODEL = [] # ALL types of models in this analysis and visualization
PATH.append("/Users/t/Desktop/CitationSpace/GoogleNet/") # papers on GoogleNet's excel sheets are stored under this folder
MODEL.append("GoogleNet")
PATH.append("/Users/t/Desktop/CitationSpace/ResNet/") # papers on ResNet's excel sheets are stored under this folder
MODEL.append("ResNet")
PATH.append("/Users/t/Desktop/CitationSpace/VGG/") # papers on VGG's excel sheets are stored under this folder
MODEL.append("VGG")
PATH.append("/Users/t/Desktop/CitationSpace/AlexNet/") # papers on AlexNet's excel sheets are stored under this folder
MODEL.append("AlexNet")
PATH.append("/Users/t/Desktop/CitationSpace/LeNet/") # papers on LeNet's excel sheets are stored under this folder
MODEL.append("LeNet")
PATH.append("/Users/t/Desktop/CitationSpace/SeNet/") # papers on SeNet's excel sheets are stored under this folder
MODEL.append("SeNet")
source = [] # sotre source article name
target = [] # store target article name
model_group = [] # store the source paper's model group
count = 0 # to count the number of files existed in the folder
for model in range(len(PATH)):
    directory_len = len(PATH[model])
    filenames = glob.glob(PATH[model] + "*.xls")
    for file in filenames:
        count = count + 1
        # Crawl individual paper's reference list into "source" and "target" arrays
        if file[-8:] == "_ref.xls":
            # read all excel references list files
            df = pd.read_excel(file)
            # get reference list's column: the name of articles
            Ref = df["Article Title"]
            new_target = 0
            for paper in Ref:
                target.append(paper)
                new_target = new_target + 1
            file_papername = file[directory_len : -8]
            for i in range(new_target):
                source.append(file_papername)
                model_group.append(MODEL[model])
        elif file[-9:] == "_cite.xls":
            # read all excel citations list files
            dc = pd.read_excel(file)
            # get citation list's column: the name of articles
            Cite = dc["Article Title"]
            new_target = 0
            for paper in Cite:
                source.append(paper)
                new_target = new_target + 1
            file_papername = file[directory_len : -9]
            for i in range(new_target):
                target.append(file_papername)
                model_group.append(MODEL[model])
print("Reference list of papers: ", len(source), len(target))
print("number of excel sheets: ", count)

Reference list of papers:  136837 136837
number of excel sheets:  7093


In [3]:
Source0 = []
Target0 = []
Model_group0 = []
len_limit = len(source)
for i in range(len_limit):
    if source[i] in target and target[i] in source:
        Source0.append(source[i])
        Target0.append(target[i])
        Model_group0.append(model_group[i])
print("Number of Entries in Current Citation Space: ", len(Source0), len(Target0), len(Model_group0))
# erase arrays no longer using
# source = []
# target = []

Number of Entries in Current Citation Space:  12680 12680 12680


In [4]:
result = sorted(zip(Source0, Target0, Model_group0))
len_limit = len(result)
num_repeat = 0
for i in range(len_limit - 1):
    if result[len_limit - 1 - i] == result[len_limit - 2 - i]:
        # Delete those citation entries that are alike
        result.pop(len_limit - 1 - i)
        num_repeat = num_repeat + 1
Source = []
Target = []
Model_group = []
for i in range(len_limit - num_repeat):
    Source.append(result[i][0])
    Target.append(result[i][1])
    Model_group.append(result[i][2])
print("Number of Entries in Current Citation Space: ", len(Source), len(Target), len(Model_group))

Number of Entries in Current Citation Space:  11698 11698 11698


In [5]:
paper = [] # Store article names
paper_model = [] # Store article's filed of corresponding model
count = [] # Store individual article's popularity (occurrence in this data base)
len_limit = len(Source)
for i in range(len_limit):
    if Source[i] not in paper:
        paper.append(Source[i])
        paper_model.append(Model_group[i])
        # to count the number of occurrences of a paper's title in both Source and Target
        count_paper = 0
        for j in range(len_limit):
            if Target[j] == Source[i]:
                count_paper = count_paper + 1
        count.append(count_paper)
    if Target[i] not in paper:
        paper.append(Target[i])
        paper_model.append(Model_group[i])
        # to count the number of occurrences of a paper's title in both Source and Target
        count_paper = 0
        for j in range(len_limit):
            if Target[j] == Target[i]:
                count_paper = count_paper + 1
        count.append(count_paper)
print("number of total papers: ", len(paper), len(count), len(paper_model))

number of total papers:  4834 4834 4834


In [6]:
len_limit = len(paper)
Weight = [] # to store the edge weight of all source and target article
for i in range(len(Source)):
    count_source = -1
    count_target = -1
    for j in range(len_limit):
        if Source[i] == paper[j]:
            count_source = count[j]
        if Target[i] == paper[j]:
            count_target = count[j]
    if count_source != -1 and count_target != -1:
        Weight.append(count_source + count_target)
    else: # print out the missing article names if any
        print(count_source, count_target)
        print(Source[i])
        print(Target[i])
print("Number of Entries in Current Citation Space: ", len(Weight))
print("Statistics about Edges' Weights")
print("Weight mean: ", np.mean(Weight))
print("Weight median: ", np.median(Weight))
print("Weight standard deviation: ", np.std(Weight))

Number of Entries in Current Citation Space:  11698
Statistics about Edges' Weights
Weight mean:  82.82552573089417
Weight median:  17.0
Weight standard deviation:  138.7333871793083


In [7]:
len_limit = len(Source)
for i in range(len_limit):
    # Cutoff citation and reference summatino number to eliminate those less popular papers' citation relationship
    cutoff = 6
    if Weight[len_limit - 1 - i] <= cutoff:
        Source.pop(len_limit - 1 - i)
        Target.pop(len_limit - 1 - i)
        Weight.pop(len_limit - 1 - i)
        Model_group.pop(len_limit - 1 - i)
print("Number of Entries in Current Citation Space: ", len(Weight))

Number of Entries in Current Citation Space:  7866


In [8]:
data = {'Source': Source, 'Target': Target, 'Weight': Weight, 'Model': Model_group} # Store result as df data type
df = pd.DataFrame(data)
df.tail(20)

,Source,Target,Weight,Model
7846,[DL] A Survey of FPGA-based Neural Network Inf...,A High Performance FPGA-based Accelerator for ...,30,AlexNet
7847,[DL] A Survey of FPGA-based Neural Network Inf...,An OpenCL(TM) Deep Learning Accelerator on Arr...,31,AlexNet
7848,[DL] A Survey of FPGA-based Neural Network Inf...,Automatic Code Generation of Convolutional Neu...,7,LeNet
7849,[DL] A Survey of FPGA-based Neural Network Inf...,Efficient and Accurate Approximations of Nonli...,55,AlexNet
7850,[DL] A Survey of FPGA-based Neural Network Inf...,Energy-Efficient CNN Implementation on a Deepl...,30,AlexNet
7851,[DL] A Survey of FPGA-based Neural Network Inf...,Frequency Domain Acceleration of Convolutional...,56,AlexNet
7852,[DL] A Survey of FPGA-based Neural Network Inf...,Frequency Domain Acceleration of Convolutional...,56,GoogleNet
7853,cPSO-CNN: An efficient PSO-based algorithm for...,Efficient Optimization of Convolutional Neural...,7,AlexNet
7854,clCaffe: OpenCL accelerated Caffe for Convolut...,Gradient-based learning applied to document re...,442,AlexNet
7855,clCaffe: OpenCL accelerated Caffe for Convolut...,ImageNet Classification with Deep Convolutiona...,371,AlexNet


In [9]:
# CITE SPACE VISUALIZATION APPROACH - 1
# A HOLISTIC VIEW: pyvis, networkx

# REFERENCES FOR PYVIS PACKAGE:
# https://pyvis.readthedocs.io/en/latest/tutorial.html#node-properties

import matplotlib.pyplot as plt

G = nx.from_pandas_edgelist(df, source = 'Source', target = 'Target', edge_attr = 'Weight')
#net = Network(notebook = True)
#net.from_nx(G)
#net.show("example.html")

# all graph options
#graphs_viz_options = [nx.draw, nx.draw_networkx, nx.draw_circular, nx.draw_kamada_kawai, nx.draw_random, nx.draw_shell, nx.draw_spring]

# plot graph option
#selected_graph_option = 0

# plot
#plt.figure(figsize=(8,6), dpi=100) 
#graphs_viz_options[selected_graph_option](G)

In [10]:
got_net = Network(height="100%", width="100%", bgcolor="#222222", font_color="white", select_menu=True, filter_menu=True)
#got_net = Network(height="100%", width="100%", bgcolor="#222222", font_color="white")
got_net.barnes_hut()
edge_data = zip(Source, Target, Weight)

for e in edge_data:
                src = e[0]
                dst = e[1]
                w = e[2]

                got_net.add_node(src, src, title = src)
                got_net.add_node(dst, dst, title = dst)
                got_net.add_edge(src, dst, value = w)

neighbor_map = got_net.get_adj_list()

# add neighbor data to node hover data
for node in got_net.nodes:
                node["title"] += " -- cited by other " + str(len(neighbor_map[node["id"]])) + " papers"
                node_size = (len(neighbor_map[node["id"]])) ** 2 # for easier distinguishibility between node sizes
                node["value"] = node_size

got_net.show("citespace.html")
#got_net.write_html("citespace.html", local=False)

In [17]:
# TEST HTML for MODEL GROUPS LABELING WITH DIFFERENT COLORS USING PYVIS

node_data = {
    "id": paper,
    "Name": paper,
    "Model": paper_model,
}
edge_data = {"source": Source, "target": Target}

nodes = pd.DataFrame(node_data)
edges = pd.DataFrame(edge_data)

# getting a group id for each company
groups = nodes.groupby("Model")["id"].apply(list).reset_index()
groups["group"] = groups.index

# finding group id for each node from groups dataframe
nodes = nodes.merge(groups, how="inner", on=["Model"])
nodes["title"] = nodes[["Name", "Model"]].apply(lambda x: f"Name:{x[0]} , Model:{x[1]}", axis=1)

nodes = nodes.drop("id_y", axis=1).set_index("id_x")

# collecting node attributes for network x
node_attrs = nodes.to_dict("index")

# creating a network x graph from dataframes
graph = nx.from_pandas_edgelist(edge_data)
nx.set_node_attributes(graph, node_attrs)


pyvis_nt = Network("1000px", "1000px", heading="Graph")
pyvis_nt.from_nx(graph)
pyvis_nt.show('test_graph.html')

In [11]:
# CITE SPACE VISUALIZATION APPROACH - 2
# PyeCharts
# REFERENCE THIS WEB PAGE for General INFO: https://zhuanlan.zhihu.com/p/38305586
# AND THIS for General INFO: https://www.heywhale.com/mw/project/5f0d5414597c0f002d5f2e09
# AND THIS for General INFO: https://gallery.pyecharts.org/#/Graph/graph_les_miserables
# AND THIS for General INFO as Well: https://pyecharts.readthedocs.io/projects/pyecharts-en/zh/latest/en-us/charts_base/#graph
# INPUT DATA STRUCTURE: https://gist.github.com/abkunal/98d35b9b235312e90f3e43c9f7b6932b
# LabelOpts Choices: https://blog.csdn.net/qq_42374697/article/details/105694022
# TitleOpts Choices: https://blog.csdn.net/qq_42374697/article/details/105612266
# LegendOpts Choices: https://blog.csdn.net/qq_42374697/article/details/105630669
# InitOpts Choices: https://blog.csdn.net/qq_42374697/article/details/105381277
# LineStyleOpts Choices: https://blog.csdn.net/qq_42374697/article/details/105694710
# THEMETYPE Choices: https://wenku.baidu.com/view/8321f8e1a900b52acfc789eb172ded630a1c9844.html

import json
from pyecharts import options as opts
from pyecharts.charts import Graph
from pyecharts.globals import ThemeType
import math

In [16]:
# Creating LINKS
OtoV = []
for i in range(len(Source)):
    OtoV.append({'source': Source[i], 'target': Target[i], 'value': Weight[i]})

# Creating NODES
Node = []
for i in range(len(paper)):
    group_position = MODEL.index(paper_model[i])
    # for Cleaner label: slicing the paper name and output in two rows
    string = paper[i]
    slices = string.split(" ") # Slice strings whenever there is " "
    former = slices[0 : round(len(slices)/2)]
    later = slices[round(len(slices)/2) : len(slices)]
    F = ""
    for f in former:
        F = F + " " + f
    L = ""
    for l in later:
        L = L + " " + l
    papername_2rows = F + " \n" + L # "\n" seperates two strings will be outputted into seperate rows
    
    Node.append({"id": paper[i], 
                 "group": group_position, 
                 "name": papername_2rows, 
                 "symbolSize": 2 * math.sqrt(count[i]), 
                 "category": paper_model[i]})

# Creating CATEGORIES
categories = []
for i in range(len(MODEL)):
    categories.append({"name": MODEL[i]})

# VISUALIZATION
set_color = "white"
c = (
    Graph(init_opts = opts.InitOpts(width = "900px",
                                    height = "800px",
                                    page_title = "Citation Space",
                                    #theme = ThemeType.DARK, # THEME
                                    # bg_color = "#000000" # Back ground color black
                                    ))
    .add(
        "",
        nodes = Node,
        links = OtoV,
        categories = categories,
        layout = "circular",
        #is_rotate_label = True,
        is_draggable = True,
        repulsion = 100,
        linestyle_opts = opts.LineStyleOpts(color = "source", 
                                            curve = 0.3,
                                            opacity = 0.7),
        label_opts = opts.LabelOpts(position = "outside", 
                                    is_show = False, 
                                    color = set_color,
                                    margin = 50,
                                    font_size = 15, 
                                    font_style = "normal",
                                    font_weight = "bold"),
    )
    .set_global_opts(
        tooltip_opts = opts.TooltipOpts(is_show = True,
                                        trigger_on = "click",
                                        formatter = "{c}"),
        #title_opts = opts.TitleOpts(title = "CITATION SPACE", 
                                    #subtitle = "-- Relationships between Papers on \n   Models Used in Computer Vision", 
                                    #pos_left = "5%",
                                    #item_gap = 20,
                                    #title_textstyle_opts = opts.TextStyleOpts(color = set_color, 
                                                                              #font_size = 40),
                                    #subtitle_textstyle_opts = opts.TextStyleOpts(color = set_color, 
                                                                                 #font_size = 16)),
        legend_opts = opts.LegendOpts(orient = "horizontal", 
                                      pos_left = "0%", 
                                      pos_top = "0%",
                                      padding = 10,
                                      item_width = 50,
                                      item_height = 25,
                                      item_gap = 20,
                                      textstyle_opts = opts.TextStyleOpts(color = set_color, 
                                                                          font_size = 20)),
    )
    .render("CiteSpace_circular.html")
)